In [20]:
#X check bug with first two in list of viewers/position being the same
#X add min/max ranking
# check code: https://github.com/bernorieder/SexCams/blob/master/timestamp_calc_all_platforms.ipynb
# churn/regulars/classficiation

import glob
import pandas as pd
from datetime import datetime

path = 'homepage_csvs/test/'
path = 'homepage_csvs/chaturbate/'

# read file names from directory
csv_files = glob.glob(path + '*.csv')

# iterate over files to create a dict of performers and their sessions
performerlist = {}
for filename in sorted(csv_files):
    
    df = pd.read_csv(filename)
    fileparts = filename.split('_')
    date_string = fileparts[2] + ' ' + fileparts[3]
    
    date = datetime.strptime(date_string, '%Y-%m-%d %H-%M-%S')
    
    for idx, values in df.iterrows():
        if values['name'] not in performerlist:
            performerlist[values['name']] = dict()
        performerlist[values['name']][date] = (values['viewers'],values['position'],values['time'])
            
#print(len(performerlist))
#print(performerlist)


In [ ]:
sessions = []
sid = 0
# days active?
# add to performer file

for name in performerlist:
    
    #if name != '':
    #   continue
       
    performer = sorted(performerlist[name].items())
    #if name == '':
    #    print(name, performer)
    #continue
    
    last_stamp = None
    counter = 0
    
    for stamp in performer:
        
        if last_stamp == None:
            
            ns = {}
            ns['id'] = sid
            sid += 1
            ns['name'] = name
            ns['start'] = stamp[0]
            ns['start_viewers'] = stamp[1][0]
            ns['start_position'] = stamp[1][1]
            ns['positions'] = []
            ns['positions'].append(stamp[1][1])
            ns['viewers'] = []
            ns['viewers'].append(stamp[1][0])
            
            last_stamp = stamp
        
        else:
            
            delta = stamp[0] - last_stamp[0]
            #print(delta.total_seconds())
            
            if delta.total_seconds() > 3700 and counter < len(performer):
                ns['end'] = last_stamp[0]
                ns['end_viewers'] = last_stamp[1][0]
                ns['end_position'] = last_stamp[1][1]
                ns['end_time'] = last_stamp[1][2]
                ns['duration'] = round((last_stamp[0] - ns['start']).total_seconds() / 60)
                ns['average_position'] = round(sum(ns['positions']) / len(ns['positions']))
                ns['variance_position'] = round(sum((i - ns['average_position']) ** 2 for i in ns['positions']) / len(ns['positions']))
                ns['max_position'] = sorted(ns['positions'])[len(ns['positions'])-1]
                ns['min_position'] = sorted(ns['positions'])[0]
                ns['average_viewers'] = round(sum(ns['viewers']) / len(ns['viewers']))
                ns['variance_viewers'] = round(sum((i - ns['average_viewers']) ** 2 for i in ns['viewers']) / len(ns['viewers']))
                ns['max_viewers'] = sorted(ns['viewers'])[len(ns['viewers'])-1]
                ns['min_viewers'] = sorted(ns['viewers'])[0]
                sessions.append(ns)
                
                ns = {}
                ns['id'] = sid
                sid += 1
                ns['name'] = name
                ns['start'] = stamp[0]
                ns['start_viewers'] = stamp[1][0]
                ns['start_position'] = stamp[1][1]
                ns['positions'] = []
                ns['positions'].append(stamp[1][1])
                ns['viewers'] = []
                ns['viewers'].append(stamp[1][0])
            
            else:
                
                ns['positions'].append(stamp[1][1])
                ns['viewers'].append(stamp[1][0])
                
            last_stamp = stamp

        if counter == len(performer)-1:
            ns['end'] = stamp[0]
            ns['end_viewers'] = stamp[1][0]
            ns['end_position'] = stamp[1][1]
            ns['end_time'] = stamp[1][2]
            ns['duration'] = round((stamp[0] - ns['start']).total_seconds() / 60)
            ns['average_position'] = round(sum(ns['positions']) / len(ns['positions']))
            ns['variance_position'] = round(sum((i - ns['average_position']) ** 2 for i in ns['positions']) / len(ns['positions']))
            ns['max_position'] = sorted(ns['positions'])[len(ns['positions'])-1]
            ns['min_position'] = sorted(ns['positions'])[0]
            ns['average_viewers'] = round(sum(ns['viewers']) / len(ns['viewers']))
            ns['variance_viewers'] = round(sum((i - ns['average_viewers']) ** 2 for i in ns['viewers']) / len(ns['viewers']))
            ns['max_viewers'] = sorted(ns['viewers'])[len(ns['viewers'])-1]
            ns['min_viewers'] = sorted(ns['viewers'])[0]
            sessions.append(ns)
            last_stamp = None

        
        counter += 1
        
df = pd.DataFrame.from_dict(sessions)
df.to_csv('sessions.csv',index=False)